# **EJERCICIO 1**

In [ ]:
import re
import torch

In [ ]:
with open("articuloIA.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 5423
Revolución en la logística: La inteligencia artificial redefine la sostenibilidad en el transporte 


In [ ]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item]
print(preprocessed[:38])

['Revolución', ' ', 'en', ' ', 'la', ' ', 'logística', ':', ' ', 'La', ' ', 'inteligencia', ' ', 'artificial', ' ', 'redefine', ' ', 'la', ' ', 'sostenibilidad', ' ', 'en', ' ', 'el', ' ', 'transporte', ' ', 'de', ' ', 'mercancías', '\n', '\n', 'Resulta', ' ', 'crucial', ' ', 'estar', ' ']


In [ ]:
print("Number of tokens:", len(preprocessed))

Number of tokens: 1715


In [ ]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

369


In [ ]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [ ]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('\n', 0)
(' ', 1)
('(', 2)
(')', 3)
(',', 4)
('.', 5)
(':', 6)
('A', 7)
('Además', 8)
('Al', 9)
('Artificial', 10)
('DB', 11)
('El', 12)
('En', 13)
('Estas', 14)
('Esto', 15)
('Hemos', 16)
('Hoy', 17)
('IA', 18)
('Inteligencia', 19)
('La', 20)
('Las', 21)
('Los', 22)
('Mediante', 23)
('No', 24)
('Otro', 25)
('Resulta', 26)
('Revolución', 27)
('Schenker', 28)
('Sin', 29)
('Somos', 30)
('Tradicionalmente', 31)
('Una', 32)
('Uno', 33)
('a', 34)
('accidentes', 35)
('adopción', 36)
('agilidad', 37)
('ajustar', 38)
('al', 39)
('algoritmos', 40)
('almacenamiento', 41)
('altamente', 42)
('alternativas', 43)
('alto', 44)
('analizar', 45)
('ante', 46)
('antes', 47)
('análisis', 48)
('aplicaciones', 49)
('aprendizaje', 50)


In [ ]:
class SimpleTokenizerV1():

    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = '''Resulta crucial estar comprometidos con la adopción y el desarrollo de soluciones de IA que mejoren la calidad de los servicios
Las empresas deben garantizar que el manejo de esta información se realice de manera ética y conforme a las normativas vigentes en materia de protección de datos
Una de las aplicaciones más destacadas de la IA en el ámbito logístico es la optimización de rutas'''

ids = tokenizer.encode(text)
ids = torch.tensor(ids).unsqueeze(0)
print(ids)

tensor([[ 26, 103, 143,  78,  80, 196,  36, 361, 127, 120, 112, 332, 112,  18,
         289, 217, 196,  64, 112, 204, 324,  21, 132, 113, 167, 289, 127, 206,
         112, 141, 183, 317, 292, 112, 207, 367, 361,  84,  34, 198, 230, 358,
         133, 210, 112, 286, 112, 111,  32, 112, 198,  49, 227, 122, 112, 196,
          18, 133, 127, 364, 203, 137, 196, 251, 112, 314]])


In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = '''IA en la logística optimiza'''
ids2 = tokenizer.encode(text)
print(ids2)
ids2 = torch.tensor(ids2).unsqueeze(0)

[18, 133, 196, 201, 250]


# **EJERCICIO 2**


In [ ]:
import torch
import torch.nn as nn

class SimpleLSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super(SimpleLSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)   # Convierte IDs en vectores
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)            # Salida: logits para cada token

    def forward(self, x):
        """
        Entrada:
            x: Tensor de forma (batch_size, sequence_length)
        Salida:
            logits: Tensor de forma (batch_size, sequence_length, vocab_size)
        """
        x = self.embedding(x)              # (batch_size, seq_len, embed_size)
        out, _ = self.lstm(x)              # (batch_size, seq_len, hidden_size)
        logits = self.fc(out)              # (batch_size, seq_len, vocab_size)
        return logits


In [ ]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context
    for _ in range(max_new_tokens):

        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]

        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)

        # Focus only on the last time step
        # (batch, n_tokens, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]

        # Apply softmax to get probabilities
        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)

        # Get the idx of the vocab entry with the highest probability value
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx

USAMOS PARA GENERAR TEXTO UNA LSTM BÁSICA CON NUESTRO VOCABULARIO GENERADO EN EL EJERCICIO 1

In [ ]:
model = SimpleLSTMModel(vocab_size, embed_size=64,hidden_size=128)

In [ ]:
output = generate_text_simple(model, idx = ids2 , max_new_tokens=20, context_size= 100)

In [ ]:
print(tokenizer.decode(output[0].tolist()))

IA en la logística optimiza términos de críticos posible Artificial destacadas cultural para ágil avanzados obstante operar operar Uno operar otros desarrollo avanzados soluciones efectiva


COMO VEMOS LA RESPUESTA ES BASTANTE MALA YA QUE EL VOCABULARIO ES MUY PEQUEÑO Y EL MODELO MUY SIMPLE. TIENDE A REPETIR PALABRAS YA QUE TAMPOCO ESTAMOS PENALIZANDO LA REPETICIÓN.

# **EJERCICIO 3-4**

AHORA VAMOS A HACER LO MISMO PERO CON LA ESTRUCTURA DE UN GPT2-124M CON CONTEXTO REDUCIDO A 256 EN LUGAR DE 1028. TAMBIÉN SE HA REDUCIDO EL TAMAÑO DEL VOCABULARIO A 50257.

In [ ]:
!pip install tiktoken

In [ ]:
from importlib.metadata import version

pkgs = ["matplotlib",
        "numpy",
        "tiktoken",
        "torch",
       ]
for p in pkgs:
    print(f"{p} version: {version(p)}")

matplotlib version: 3.10.0
numpy version: 2.0.2
tiktoken version: 0.9.0
torch version: 2.6.0+cu124


In [ ]:
import torch.nn as nn


class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift


class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))


class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)


class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        # Shape: (b, num_tokens, d_out)
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec


class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):

        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x



class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [ ]:
import torch

GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval();  # Disable dropout during inference

In [ ]:
import tiktoken

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

In [ ]:
start_context = 'IA en la logística optimiza'
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 IA en la logística optimizaesh modifierificentatonfiguredlevision TTarianFree foundation


LA RESPUESTA SIGUE SIENDO BASTANTE MALA PORQUE EL MODELO NO HA SIDO ENTRENADO AÚN

# EJERCICIO 5

In [ ]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True, num_workers=0):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader

In [ ]:
GPT_CONFIG_124M

{'vocab_size': 50257,
 'context_length': 256,
 'emb_dim': 768,
 'n_heads': 12,
 'n_layers': 12,
 'drop_rate': 0.1,
 'qkv_bias': False}

En este caso hemos dejado el train_ratio en 0.80 ya que a 0.90 quedaba una validación muy pequeña que producía un NAN.

In [ ]:
# Train/validation ratio
train_ratio = 0.80
split_idx = int(train_ratio * len(raw_text))
train_data = raw_text[:split_idx]
val_data = raw_text[split_idx:]


torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [ ]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss


def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) # no assignment model = model.to(device) necessary for nn.Module classes


torch.manual_seed(123) # For reproducibility due to the shuffling in the data loader

with torch.no_grad(): # Disable gradient tracking for efficiency because we are not training, yet
    train_loss = calc_loss_loader(train_loader, model, device)
    val_loss = calc_loss_loader(val_loader, model, device)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)

Training loss: 10.980511665344238
Validation loss: 11.019054412841797


In [ ]:

def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate_text_simple(
            model=model, idx=encoded,
            max_new_tokens=50, context_size=context_size
        )
        decoded_text = token_ids_to_text(token_ids, tokenizer)
        print(decoded_text.replace("\n", " "))  # Compact print format
    model.train()

In [ ]:
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer):
    # Initialize lists to track losses and tokens seen
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode

        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward() # Calculate loss gradients
            optimizer.step() # Update model weights using loss gradients
            tokens_seen += input_batch.numel()
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Print a sample text after each epoch
        generate_and_print_sample(
            model, tokenizer, device, start_context
        )

    return train_losses, val_losses, track_tokens_seen

In [ ]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

num_epochs = 10
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="IA en la logística optimiza", tokenizer=tokenizer
)

Ep 1 (Step 000000): Train loss 9.572, Val loss 9.670
IA en la logística optimiza de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de
IA en la logística optimiza, la IA, la IA en la IA la IA en la IA, la de la IA la IA en la IA de la IA de la IA en la IA, la IA de la IA en la IA, la IA en la IAes de
Ep 3 (Step 000005): Train loss 4.930, Val loss 7.826
IA en la logística optimiza el de la IA en la IA en la IA en la IA de la IA en la IA en la IA en la IA en la IA de la IA en la IA en la IA en la IA en la IA de la IA de la IA de
IA en la logística optimizaos, la IA en la IA en la IA en la IA en la IA en la IA en la IA en la IA en la IA en la IA en la IA en la IA en la IA en la IA en la IA en la IA en
IA en la logística optimizaas. Esto de rutas, la IA en la IA en el de la IA en la IA en la IA en la IA en la IA en la IA en tiempo real de la IA en la IA en la IAes de
Ep 6 (Step 000010): Train lo

COMO VEMOS LA SOLUCIÓN MEJORA NOTABLEMENTE A MEDIDA QUE PASAN LAS EPOCHS, HEMOS PODIDO REALIZAR EL ENTRENAMIENTO YA QUE EL TEXTO ES BREVE. PERO DADO EL TAMAÑO DE NUESTRO TEXTO LA SOLUCIÓN NO ES DEL TODO BUENA.

# EJERCICIO 6

In [ ]:
from transformers import pipeline

model_name = "deepset/roberta-base-squad2"

# Crear un pipeline de question answering
qa_pipeline = pipeline("question-answering", model=model_name)

# Contexto y pregunta

contexto = """
PSG (Paris Saint-Germain) goleó 5-0 a Inter la gran final y se consagró campeón de la UEFA Champions League 2025, en el Allianz Arena de Múnich en una exhibición de fútbol para el recuerdo que significó la goleada más abultada en un partido de este calibre.
El equipo de Luis Enrique obtuvo su primera corona en Europa gracias a los goles de Achraf Hakimi, Désiré Doué (en dos ocasiones), Khvicha Kvaratskhelia y Senny Mayulu en una demostración futbolística que sin lugar a dudas quedará en los libros de historia.
"""

pregunta = "¿Quién ganó la Champions League 2025?"

# Realizar la tarea de question answering
respuesta = qa_pipeline(question=pregunta, context=contexto)

# Mostrar los resultados
print(f"Pregunta: {pregunta}")
print(f"Respuesta: {respuesta['answer']}")
print(f"Puntuación: {respuesta['score']:.4f}")
print(f"Inicio: {respuesta['start']}")
print(f"Fin: {respuesta['end']}")

Device set to use cpu


Pregunta: ¿Quién ganó la Champions League 2025?
Respuesta: Paris Saint-Germain
Puntuación: 0.0495
Inicio: 6
Fin: 25


In [ ]:
from transformers import pipeline

# Crear un pipeline de question answering
qa_pipeline = pipeline("question-answering", model="MMG/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es-finetuned-sqac")

# Contexto y pregunta
contexto = """
 PSG (Paris Saint-Germain) goleó 5-0 a Inter la gran final y se consagró campeón de la UEFA Champions League 2025, en el Allianz Arena de Múnich en una exhibición de fútbol para el recuerdo que significó la goleada más abultada en un partido de este calibre.
El equipo de Luis Enrique obtuvo su primera corona en Europa gracias a los goles de Achraf Hakimi, Désiré Doué (en dos ocasiones), Khvicha Kvaratskhelia y Senny Mayulu en una demostración futbolística que sin lugar a dudas quedará en los libros de historia.
"""

pregunta = "¿Quién ganó la Champions League 2025?"

# Realizar la tarea de question answering
respuesta = qa_pipeline(question=pregunta, context=contexto)

# Mostrar los resultados
print(f"Pregunta: {pregunta}")
print(f"Respuesta: {respuesta['answer']}")
print(f"Puntuación: {respuesta['score']:.4f}")
print(f"Inicio: {respuesta['start']}")
print(f"Fin: {respuesta['end']}")

Device set to use cpu


Pregunta: ¿Quién ganó la Champions League 2025?
Respuesta: PSG
Puntuación: 0.8419
Inicio: 2
Fin: 5


In [ ]:
from transformers import pipeline

# Crear un pipeline de question answering
qa_pipeline = pipeline("question-answering", model="MMG/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es-finetuned-sqac")

# Contexto y pregunta
contexto = """
 PSG (Paris Saint-Germain) goleó 5-0 a Inter la gran final y se consagró campeón de la UEFA Champions League 2025, en el Allianz Arena de Múnich en una exhibición de fútbol para el recuerdo que significó la goleada más abultada en un partido de este calibre.
El equipo de Luis Enrique obtuvo su primera corona en Europa gracias a los goles de Achraf Hakimi, Désiré Doué (en dos ocasiones), Khvicha Kvaratskhelia y Senny Mayulu en una demostración futbolística que sin lugar a dudas quedará en los libros de historia.
"""

pregunta = "¿Cual fue el resultado de la final de la Champions 2025?"
# Realizar la tarea de question answering
respuesta = qa_pipeline(question=pregunta, context=contexto)

# Mostrar los resultados
print(f"Pregunta: {pregunta}")
print(f"Respuesta: {respuesta['answer']}")
print(f"Puntuación: {respuesta['score']:.4f}")
print(f"Inicio: {respuesta['start']}")
print(f"Fin: {respuesta['end']}")

Device set to use cpu


Pregunta: ¿Cual fue el resultado de la final de la Champions 2025?
Respuesta: 5-0
Puntuación: 0.3084
Inicio: 34
Fin: 37


Como se observa ambos modelos han dado la respuesta correcta. Si bien es verdad que el primero dado que el idioma es el inglés el score que tiene, es decir la seguridad en la respuesta es muy baja. En cambio en el modelo utilizado como ejemplo en clase que tiene un corpus en español el score es mucho más alto.